In [1]:
import alpaca_trade_api as tradeapi
import numpy as np
import pandas as pd
import talib as ta
import json
import tqdm.notebook as tqdm

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
key_id = None
secret_key = None
with open('alpakey') as key_file:
    keys = key_file.readlines()
    key_id = keys[0].strip()
    secret_key = keys[1].strip()
    
api_url = "https://paper-api.alpaca.markets"

alpaca = tradeapi.REST(key_id, secret_key, api_url, api_version='v2')

In [4]:
symbols = ['A', 'AA', 'AAL', 'AAPL', 'ABBV', 'ABC', 'ABEV', 'ABT', 'ACB',
       'ACGL', 'ACM', 'ACN', 'ACWI', 'ACWX', 'ADBE', 'ADI', 'ADNT', 'ADP',
       'ADSK', 'ADT', 'AEO', 'AEP', 'AER', 'AES', 'AFL', 'AGG', 'AGN',
       'AGNC', 'AIG', 'AL', 'ALK', 'ALLY', 'ALXN', 'AM', 'AMAT', 'AMC',
       'AMCR', 'AMD', 'AME', 'AMGN', 'AMH', 'AMJ', 'AMLP', 'AMRN', 'AMT',
       'AMTD', 'AMX', 'AMZN', 'ANF', 'ANGI', 'ANTM', 'AON', 'AOS', 'APA',
       'APLE', 'APO', 'APT', 'APTV', 'APY', 'AR', 'ARCC', 'ARMK', 'ARNC',
       'ASHR', 'ATH', 'ATI', 'ATUS', 'ATVI', 'AUY', 'AVGO', 'AVTR',
       'AVYA', 'AXL', 'AXP', 'AXTA', 'AYTU', 'AZN', 'BA', 'BABA', 'BAC',
       'BAX', 'BB', 'BBBY', 'BBD', 'BBEU', 'BBVA', 'BBY', 'BCRX', 'BCS',
       'BDN', 'BDX', 'BE', 'BEN', 'BERY', 'BGCP', 'BGS', 'BHC', 'BHP',
       'BIDU', 'BIL', 'BILI', 'BIOC', 'BJ', 'BK', 'BKD', 'BKLN', 'BKR',
       'BLDR', 'BLL', 'BLMN', 'BMY', 'BMY^', 'BND', 'BNDX', 'BP', 'BPY',
       'BRFS', 'BRK.B', 'BRX', 'BSX', 'BTG', 'BTI', 'BTU', 'BUD', 'BWA',
       'BX', 'BXMT', 'BYD', 'C', 'CAG', 'CAH', 'CAKE', 'CAR', 'CARG',
       'CAT', 'CB', 'CBRE', 'CC', 'CCC', 'CCEP', 'CCI', 'CCJ', 'CCK',
       'CCL', 'CCO', 'CDAY', 'CDE', 'CDEV', 'CDNS', 'CERN', 'CF', 'CFG',
       'CFX', 'CG', 'CHK', 'CHKP', 'CHNG', 'CHRW', 'CHS', 'CHTR', 'CHWY',
       'CI', 'CIM', 'CIT', 'CL', 'CLB', 'CLDR', 'CLF', 'CLNY', 'CLR',
       'CLVS', 'CMA', 'CMCSA', 'CME', 'CNC', 'CNDT', 'CNK', 'CNP', 'CNQ',
       'CNX', 'CODX', 'COF', 'COG', 'COLD', 'COMM', 'CONE', 'COP', 'COTY',
       'COUP', 'CPB', 'CPE', 'CPRI', 'CPRT', 'CRC', 'CRM', 'CRON', 'CROX',
       'CRWD', 'CSCO', 'CSPR', 'CSTM', 'CSX', 'CTL', 'CTSH', 'CTVA',
       'CTXS', 'CUBE', 'CVE', 'CVET', 'CVNA', 'CVS', 'CVX', 'CX', 'CXO',
       'CY', 'CYH', 'CZR', 'D', 'DAL', 'DB', 'DBX', 'DCP', 'DD', 'DDOG',
       'DE', 'DEAC', 'DELL', 'DFS', 'DG', 'DGX', 'DHI', 'DHR', 'DHT',
       'DIS', 'DISCA', 'DISCK', 'DISH', 'DK', 'DKS', 'DLR', 'DLTR', 'DNR',
       'DO', 'DOC', 'DOCU', 'DOW', 'DRE', 'DRH', 'DRI', 'DT', 'DUK',
       'DUST', 'DVN', 'DWT', 'DXC', 'EA', 'EAF', 'EAT', 'EB', 'EBAY',
       'ED', 'EEM', 'EFA', 'EIX', 'EL', 'ELAN', 'EMB', 'EMR', 'ENB',
       'ENBL', 'ENDP', 'ENLC', 'ENPH', 'EOG', 'EPD', 'EPR', 'EPRT', 'EQH',
       'EQM', 'EQNR', 'EQR', 'EQT', 'ERI', 'ERIC', 'ERX', 'ESI', 'ESTC',
       'ET', 'ETFC', 'ETN', 'ETRN', 'ETSY', 'EURN', 'EVRI', 'EWBC', 'EWG',
       'EWH', 'EWJ', 'EWT', 'EWY', 'EWZ', 'EXAS', 'EXC', 'EXEL', 'EXPD',
       'EXPE', 'EZU', 'F', 'FANG', 'FAST', 'FB', 'FCAU', 'FCEL', 'FCX',
       'FDX', 'FE', 'FEYE', 'FEZ', 'FG', 'FG+', 'FHN', 'FIS', 'FISV',
       'FIT', 'FITB', 'FIVE', 'FL', 'FLEX', 'FLR', 'FND', 'FNF', 'FOLD',
       'FOSL', 'FOXA', 'FSCT', 'FSLY', 'FTCH', 'FTI', 'FTSV', 'FTV',
       'FWONK', 'FXI', 'GCI', 'GD', 'GDDY', 'GDS', 'GDX', 'GDXJ', 'GE',
       'GEL', 'GFI', 'GGB', 'GILD', 'GIS', 'GKOS', 'GLD', 'GLPI', 'GLUU',
       'GLW', 'GM', 'GME', 'GNW', 'GOGO', 'GOLD', 'GOOGL', 'GOOS', 'GOVT',
       'GPK', 'GPN', 'GPOR', 'GPRO', 'GPS', 'GRPN', 'GRUB', 'GS', 'GSK',
       'GT', 'GUSH', 'H', 'HAL', 'HAS', 'HBAN', 'HBI', 'HCA', 'HD', 'HDB',
       'HES', 'HFC', 'HIG', 'HL', 'HLT', 'HLX', 'HMY', 'HOG', 'HOLX',
       'HOME', 'HON', 'HP', 'HPE', 'HPP', 'HPQ', 'HRB', 'HRL', 'HSBC',
       'HSIC', 'HST', 'HTA', 'HTHT', 'HTZ', 'HUM', 'HUN', 'HUYA', 'HYAC',
       'HYG', 'I', 'IAA', 'IAG', 'IAU', 'IBIO', 'IBM', 'IBN', 'ICE',
       'IEF', 'IEFA', 'IEMG', 'IGT', 'IIVI', 'IJR', 'IMGN', 'INCY',
       'INDA', 'INFN', 'INFO', 'INFY', 'ING', 'INO', 'INTC', 'INVH',
       'IOVA', 'IP', 'IPG', 'IQ', 'IQV', 'IR', 'IRM', 'ISBC', 'ITUB',
       'IVR', 'IVV', 'IVZ', 'IWM', 'IXUS', 'IYR', 'JBLU', 'JCI', 'JCP',
       'JD', 'JDST', 'JEF', 'JNJ', 'JNK', 'JNPR', 'JNUG', 'JPM', 'JWN',
       'K', 'KAR', 'KBH', 'KDMN', 'KDP', 'KEY', 'KEYS', 'KGC', 'KHC',
       'KIM', 'KKR', 'KMI', 'KMX', 'KNX', 'KO', 'KOS', 'KR', 'KRE', 'KSS',
       'LB', 'LBTYK', 'LEN', 'LIN', 'LK', 'LKQ', 'LLY', 'LM', 'LNC',
       'LNG', 'LOW', 'LPI', 'LPX', 'LQD', 'LRCX', 'LSCC', 'LTHM', 'LULU',
       'LUV', 'LVGO', 'LVS', 'LXP', 'LYB', 'LYFT', 'LYG', 'LYV', 'M',
       'MA', 'MAC', 'MAR', 'MAS', 'MAT', 'MBT', 'MCD', 'MCHP', 'MCK',
       'MD', 'MDLA', 'MDLZ', 'MDRX', 'MDT', 'MEET', 'MET', 'MFA', 'MGM',
       'MGP', 'MGY', 'MIK', 'MLCO', 'MLPA', 'MMC', 'MMM', 'MMP', 'MNK',
       'MNST', 'MO', 'MOMO', 'MOS', 'MPC', 'MPLX', 'MPW', 'MRK', 'MRNA',
       'MRO', 'MRVL', 'MS', 'MSFT', 'MTCH', 'MTDR', 'MTG', 'MU', 'MUFG',
       'MUR', 'MXIM', 'MYL', 'NAVI', 'NBL', 'NBR', 'NCLH', 'NCR', 'NEE',
       'NEM', 'NET', 'NEX', 'NFLX', 'NGL', 'NI', 'NIO', 'NKE', 'NLOK',
       'NLSN', 'NLY', 'NOG', 'NOK', 'NOMD', 'NOV', 'NOW', 'NRG', 'NRZ',
       'NSC', 'NTAP', 'NTNX', 'NTR', 'NUAN', 'NUGT', 'NVDA', 'NVST',
       'NVTA', 'NWL', 'NWSA', 'NXPI', 'NYCB', 'NYMT', 'NYT', 'O', 'OAS',
       'OC', 'ODP', 'OHI', 'OI', 'OIBR.C', 'OIH', 'OII', 'OKE', 'OLLI',
       'OLN', 'OMC', 'OMI', 'ON', 'ONEM', 'OPK', 'ORCL', 'OSW', 'OUT',
       'OVV', 'OXY', 'PAA', 'PACB', 'PAGP', 'PAGS', 'PANW', 'PAYX',
       'PBCT', 'PBF', 'PBI', 'PBR', 'PCAR', 'PCG', 'PD', 'PDCE', 'PDCO',
       'PDD', 'PE', 'PEAK', 'PEB', 'PEG', 'PEI', 'PENN', 'PEP', 'PFE',
       'PFF', 'PFGC', 'PG', 'PGR', 'PGRE', 'PGX', 'PHM', 'PINS', 'PK',
       'PLAN', 'PLD', 'PLNT', 'PLUG', 'PM', 'PNC', 'PPD', 'PPG', 'PPL',
       'PRSP', 'PRTY', 'PRU', 'PS', 'PSEC', 'PSTG', 'PSX', 'PTEN', 'PTON',
       'PUMP', 'PXD', 'PYPL', 'QCOM', 'QD', 'QEP', 'QGEN', 'QQQ', 'QRTEA',
       'QSR', 'QTT', 'RCL', 'RDFN', 'RDS.A', 'RDS.B', 'REAL', 'RES',
       'REYN', 'REZI', 'RF', 'RIG', 'RLGY', 'RLJ', 'ROIC', 'ROKU', 'ROST',
       'RPAI', 'RRC', 'RRR', 'RSX', 'RTN', 'RUN', 'RWT', 'S', 'SABR',
       'SAN', 'SAVE', 'SBGI', 'SBH', 'SBRA', 'SBSW', 'SBUX', 'SC', 'SCHE',
       'SCHF', 'SCHW', 'SDC', 'SDS', 'SE', 'SEAS', 'SERV', 'SFIX', 'SFM',
       'SGMS', 'SH', 'SHAK', 'SHO', 'SHV', 'SHY', 'SIG', 'SIRI', 'SIX',
       'SJNK', 'SKT', 'SKX', 'SLB', 'SLCA', 'SLM', 'SLV', 'SM', 'SMAR',
       'SMFG', 'SMH', 'SNAP', 'SO', 'SONO', 'SPCE', 'SPDW', 'SPG', 'SPGI',
       'SPLG', 'SPLK', 'SPOT', 'SPR', 'SPTS', 'SPWR', 'SPXL', 'SPXS',
       'SPXU', 'SPY', 'SQ', 'SQQQ', 'SRE', 'SRNE', 'SSNC', 'STAY', 'STLD',
       'STNE', 'STOR', 'STT', 'STWD', 'STX', 'SU', 'SUM', 'SVMK', 'SWK',
       'SWKS', 'SWN', 'SYF', 'SYK', 'SYY', 'T', 'TAK', 'TAL', 'TAP',
       'TCO', 'TCOM', 'TDC', 'TDOC', 'TEAM', 'TECK', 'TEL', 'TELL',
       'TEVA', 'TFC', 'TGE', 'TGNA', 'TGT', 'THC', 'TIF', 'TIP', 'TJX',
       'TLRY', 'TLT', 'TME', 'TMHC', 'TMO', 'TMUS', 'TNXP', 'TOL', 'TOT',
       'TPH', 'TPR', 'TQQQ', 'TRGP', 'TRIL', 'TRIP', 'TRMB', 'TROW',
       'TRU', 'TRV', 'TS', 'TSCO', 'TSG', 'TSLA', 'TSM', 'TSN', 'TTWO',
       'TUP', 'TVTY', 'TWLO', 'TWO', 'TWTR', 'TXMD', 'TXN', 'TXRH', 'TXT',
       'UA', 'UAA', 'UAL', 'UBER', 'UBS', 'UCO', 'UNH', 'UNIT', 'UNM',
       'UNP', 'UNVR', 'UPS', 'UPWK', 'URBN', 'USB', 'USFD', 'USO', 'UTX',
       'UVXY', 'UWT', 'V', 'VAL', 'VALE', 'VCSH', 'VEA', 'VEON', 'VER',
       'VEU', 'VFC', 'VG', 'VGK', 'VIAC', 'VIAV', 'VICI', 'VIPS', 'VLO',
       'VNO', 'VNOM', 'VNQ', 'VOD', 'VOO', 'VOYA', 'VRT', 'VST', 'VT',
       'VTEB', 'VTI', 'VTR', 'VWO', 'VXRT', 'VXX', 'VZ', 'W', 'WAB',
       'WBA', 'WBT', 'WDAY', 'WDC', 'WELL', 'WEN', 'WES', 'WETF', 'WFC',
       'WH', 'WLL', 'WM', 'WMB', 'WMGI', 'WMT', 'WORK', 'WPG', 'WPX',
       'WRK', 'WTI', 'WTRH', 'WU', 'WW', 'WWE', 'WY', 'WYNN', 'X', 'XBI',
       'XEC', 'XEL', 'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK', 'XLNX',
       'XLP', 'XLRE', 'XLU', 'XLV', 'XLY', 'XOG', 'XOM', 'XOP', 'XPO',
       'XRAY', 'XRT', 'XRX', 'YNDX', 'YPF', 'YUM', 'YUMC', 'Z', 'ZBH',
       'ZEN', 'ZION', 'ZM', 'ZNGA', 'ZS', 'ZTO', 'ZTS', 'ZVZZT']

In [5]:
barsets = []

cutoff_date = '2020-02-06T12:50:00-05:00'

for i in range(0, len(symbols), 100):
    barsets.append(alpaca.get_barset(symbols[i:i+100], '15Min', 1000, until=cutoff_date).df.stack(level=0).reset_index())
    barsets.append(alpaca.get_barset(symbols[i:i+100], '15Min', 1000, after=cutoff_date).df.stack(level=0).reset_index())

In [6]:
all_stocks = pd.concat(barsets)

In [7]:
all_stocks.columns = ['date', 'symbol', 'close', 'high', 'low', 'open', 'volume']

In [8]:
stocks_by_symbol = all_stocks.groupby('symbol')

In [9]:
all_stocks[['close', 'high', 'low', 'open', 'volume']] = all_stocks[['close', 'high', 'low', 'open', 'volume']] \
                                                            .apply(pd.to_numeric, errors='coerce')

In [10]:
top_stocks = all_stocks.set_index('symbol').loc[stocks_by_symbol.volume.mean().nlargest(1000).index]

In [11]:
top_stocks.to_hdf('../Data/Clean/intraday.h5', key='intraday')